# Moduły

In [ ]:
from google.colab import drive

import zipfile
import os

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
import cv2
from numpy import ndarray
from brisque import BRISQUE
from skimage.metrics import structural_similarity as ssim

In [ ]:
from datetime import datetime

# Stałe

In [ ]:
TRAIN_DATASET_FOLDER = '/content/train_dataset'
VAL_DATASET_FOLDER = '/content/val_dataset'
TEST_DATASET_FOLDER = '/content/test_dataset'
REFERENCE_DATASET_FOLDER = '/content/reference_dataset'
INPUT_97_DATASET_FOLDER = '/content/input_97_dataset'

MAIN_FOLDER = '/content/drive/MyDrive/'

# Funkcje pomocnicze

## Rozpakowanie archiwum

In [ ]:
def unzip(source_path, target_path, with_main_folder=True):
  if os.path.exists(source_path):
    os.makedirs(target_path, exist_ok=True)

    with zipfile.ZipFile(source_path, 'r') as zip_ref:
      if with_main_folder:
        zip_ref.extractall(target_path)
      else:
        for content in zip_ref.namelist():
          name_archive = content.split('/', 1)[0]
          content = content.split('/', 1)[-1] if '/' in content else content

          if content == '' or content == '/':
            continue

          with zip_ref.open(name_archive + '/' + content) as source, open(os.path.join(target_path, content), "wb") as target:
            target.write(source.read())

    print(f'Content unziped: source={source_path}, target={target_path}')
  else:
    print(f'Cannot unzip: source={source_path}, target={target_path}')

## Operacje na plikach EXR

In [ ]:
 # enable using OpenEXR with OpenCV
 os.environ['OPENCV_IO_ENABLE_OPENEXR'] = "1"

In [ ]:
def read_exr(im_path: str)-> ndarray:
  return cv2.imread(
  filename=im_path,
  flags=cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH
  )

In [ ]:
def tone_map_reinhard(image: ndarray)-> ndarray:
  tonemap_operator = cv2.createTonemapReinhard(
  gamma=2.2,
  intensity=0.0,
  light_adapt=0.0,
  color_adapt=0.0
  )
  result = tonemap_operator.process(src=image)
  return result

In [ ]:
def tone_map_mantiuk(image: ndarray)-> ndarray:
  tonemap_operator = cv2.createTonemapMantiuk(
  gamma=2.2,
  scale=0.85,
  saturation=1.2
  )
  result = tonemap_operator.process(src=image)
  return result

## Metryki

### BRISQUE

In [ ]:
def evaluate_image(image: ndarray)-> float:
 metric = BRISQUE(url=False)
 return metric.score(img=image)

# Wczytanie danych

In [ ]:
drive.mount('/content/drive')

In [ ]:
data_folder = MAIN_FOLDER + '/data/prepared'

In [ ]:
print(f"Zawartość folderu {data_folder}: {os.listdir(data_folder)}")

In [ ]:
data_folder_contents = os.listdir(data_folder)

for file in data_folder_contents:
  print(f"- \"{file}\"")
  if 'val' in file.lower():
    unzip(data_folder + '/' + file, VAL_DATASET_FOLDER, False)
  elif 'train' in file.lower():
    unzip(data_folder + '/' + file, TRAIN_DATASET_FOLDER, False)
  elif 'test' in file.lower():
    unzip(data_folder + '/' + file, TEST_DATASET_FOLDER, False)
  elif 'reference' in file.lower():
    unzip(data_folder + '/' + file, REFERENCE_DATASET_FOLDER, False)
  elif 'input_97' in file.lower():
    unzip(data_folder + '/' + file, INPUT_97_DATASET_FOLDER, False)

# Przygotowanie danych

# Model

## Architektura sieci

## Trening

## Test

## Zapis modelu

In [ ]:
now = datetime.now()

model_name = f'model_{now.month}_{now.day}_{now.hour}_{now.minute}_{now.second}.pth'